<a href="https://colab.research.google.com/github/alexmarcel/neural-networks/blob/main/simple_classifier_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install torchvision for GPU
!pip install torch torchvision pandas matplotlib

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time

In [ ]:
# Chose device, GPU if exist, if not use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Check device selected
print(f'Using device: {device}\n')

# Check GPU device details
torch.cuda.is_available()
!nvidia-smi

In [ ]:
# Create a Model Class that inherits nn.Module

class Model(nn.Module):
  # input layer (number of features of the dataset) -->
  # hidden layer 1 (number of neurons) -->
  # hidden layer 2 (number of neurons) -->
  # hidden layer ... (number of ...) -->
  # output (labels)

  def __init__(self, in_features=4, h1=128, h2=64, h3=32, h4=16, out_features=3):
    super().__init__() # Instantiate nn module
    self.hl1 = nn.Linear(in_features, h1)
    self.hl2 = nn.Linear(h1, h2)
    self.hl3 = nn.Linear(h2, h3)
    self.hl4 = nn.Linear(h3, h4)
    self.out = nn.Linear(h4, out_features)

  def forward(self, x):
    x = F.relu(self.hl1(x))
    x = F.relu(self.hl2(x))
    x = F.relu(self.hl3(x))
    x = F.relu(self.hl4(x))
    x = self.out(x)

    return x

In [ ]:
# Manual seed for randomization
torch.manual_seed(10)

# Create an instance of model
model = Model().to(device) # Move model to selected device (GPU or CPU)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Load dataset
my_dataframe = pd.read_csv('iris.csv')

# Change last column strings to number
my_dataframe['species'] = my_dataframe['species'].replace('Iris-setosa', 0.0)
my_dataframe['species'] = my_dataframe['species'].replace('Iris-versicolor', 1.0)
my_dataframe['species'] = my_dataframe['species'].replace('Iris-virginica', 2.0)

# Check data table
my_dataframe

In [ ]:
# Check dataset shape (records, features)
my_dataframe.shape

In [ ]:
# Check for null values in dataset
my_dataframe.isnull().values.any()

In [ ]:
# Check dataset correlation
my_dataframe.corr()

In [ ]:
# Set X features, y labels
X = my_dataframe.drop('species', axis=1)  # Select features, drop label column into X
y = my_dataframe['species']               # Select only label column into y

# Convert to numpy arrays
X = X.values
y = y.values

# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# Convert X features to float tensors
X_train = torch.FloatTensor(X_train).to(device)  # Move train features data to selected device (GPU or CPU)
X_test = torch.FloatTensor(X_test).to(device)    # Move train features test to selected device (GPU or CPU)

# Convert y labels to float tensors
y_train = torch.LongTensor(y_train).to(device)   # Move train label  data to selected device (GPU or CPU)
y_test = torch.LongTensor(y_test).to(device)     # Move train label data to selected device (GPU or CPU)

# Set criterion of model to measure the error, how far off the predictions are from the data
criterion = nn.CrossEntropyLoss()

# Choose Adam Optimizer, lr = learning rate (if error doesn't go down after a bunch of iterations (epochs), lower our learning rate)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Check model
model.parameters

In [ ]:
# Readjust learning rate if retraining
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train model
# Epoch? (one run thru all the training data in the neural network)
epochs = 1000
losses = []
start_time = time.time()

for i in range(epochs):
  # Go forward and get a prediction
  y_pred = model.forward(X_train) # Get predicted results

  # Measure the loss/error
  loss = criterion(y_pred, y_train) # Predicted value vs y_train value

  # Loss tracking
  losses.append(loss.detach().cpu().numpy()) # Added .cpu() for GPU loss tracking, remove if using CPU

  # Print every 10 epoch
  if i%10 == 0:
    print(f'Epoch : {i} and loss : {loss}')

  # Do back propagation: take error rate of forward propagation and feed it back thru the neural network to fine tune the weights
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nElapsed Time: {elapsed_time:.2f} seconds")

In [ ]:
# Create graph
plt.plot(range(epochs), losses)
plt.ylabel("Loss/Error")
plt.xlabel("Epoch")

In [ ]:
# Evaluate model on test data set
with torch.no_grad(): # Basically turn off back propagation
  y_eval = model.forward(X_test) # X_test are features from test set, y_eval will be predictions
  loss = criterion(y_eval, y_test) # Find the loss or error

  print(loss)

In [ ]:
# Evaluate 2
correct = 0

with torch.no_grad():
  for i, data in enumerate(X_test):
    y_val = model.forward(data)

    # Will tell what result class neural network think it is
    print(f'{i+1}) \t {str(y_val)} \t\t {y_test[i]} \t {y_val.argmax().item()}')

    # Correct or not
    if y_val.argmax().item() == y_test[i]:
      correct +=1

# Percentage correct
correct_percent = (correct/len(y_test)) * 100

print(f'\n{correct}/{len(y_test)} correct predictions ({round(correct_percent, 2)}%)')

In [ ]:
# Save MODEL
torch.save(model.state_dict(), 'mymodel.pt')

In [ ]:
# Load MODEL
new_model = Model()
new_model.load_state_dict(torch.load('mymodel.pt'))

# Make sure model is loaded correctly
new_model.eval()

In [ ]:
# Test new data
new_data = torch.tensor([4, 3, 1, 0.2]) # Input test features

with torch.no_grad():
  print(f'{new_model(new_data)} {new_model(new_data).argmax().item()}')